In [1]:
from example import f_m_func

f_m = f_m_func()
print(f_m)

#f_m is a function for a specific measurement, which connects *_sp sensor pose with land mark observaion *_lm

-log(sigma_d) - log(sigma_fa) - log(x_lm**2 - 2*x_lm*x_sp + x_sp**2 + y_lm**2 - 2*y_lm*y_sp + y_sp**2)/2 - log(pi) - 0.693147180559946 - 0.5*(sigma_d**2*(mu_fa - atan2((-x_lm + x_sp)*sin(yaw_sp) + (y_lm - y_sp)*cos(yaw_sp), (2*(x_lm - x_sp)*cos(yaw_sp)**2 + (y_lm - y_sp)*sin(2*yaw_sp))/(2*cos(yaw_sp))))**2 + sigma_fa**2*(mu_d - sqrt(x_lm**2 - 2*x_lm*x_sp + x_sp**2 + y_lm**2 - 2*y_lm*y_sp + y_sp**2))**2)/(sigma_d**2*sigma_fa**2)


In [2]:
from example import eq_factory, unify_eqs

#we have observation from 2 distinct sensor poses
sp0, sp1 = eq_factory(f_m, ['x_sp', 'y_sp', 'yaw_sp'], idx=range(2))

#sensor pose 0 observes landmark 0 - 2
sp0 = eq_factory(sp0, ['x_lm', 'y_lm'], idx=range(3))

#sensor pose 1 observes landmark 0 - 2
sp1 = eq_factory(sp1, ['x_lm', 'y_lm'], idx=range(3))

#make each uq a unique measurement
eqs = unify_eqs(sp0 + sp1, syms=['mu_fa', 'mu_d'])

for eq in eqs:
    print ('----')
    print (eq)

----
-log(sigma_d) - log(sigma_fa) - log(x_lm_0**2 - 2*x_lm_0*x_sp_0 + x_sp_0**2 + y_lm_0**2 - 2*y_lm_0*y_sp_0 + y_sp_0**2)/2 - log(pi) - 0.693147180559946 - 0.5*(sigma_d**2*(mu_fa_0 - atan2((-x_lm_0 + x_sp_0)*sin(yaw_sp_0) + (y_lm_0 - y_sp_0)*cos(yaw_sp_0), (2*(x_lm_0 - x_sp_0)*cos(yaw_sp_0)**2 + (y_lm_0 - y_sp_0)*sin(2*yaw_sp_0))/(2*cos(yaw_sp_0))))**2 + sigma_fa**2*(mu_d_0 - sqrt(x_lm_0**2 - 2*x_lm_0*x_sp_0 + x_sp_0**2 + y_lm_0**2 - 2*y_lm_0*y_sp_0 + y_sp_0**2))**2)/(sigma_d**2*sigma_fa**2)
----
-log(sigma_d) - log(sigma_fa) - log(x_lm_1**2 - 2*x_lm_1*x_sp_0 + x_sp_0**2 + y_lm_1**2 - 2*y_lm_1*y_sp_0 + y_sp_0**2)/2 - log(pi) - 0.693147180559946 - 0.5*(sigma_d**2*(mu_fa_1 - atan2((-x_lm_1 + x_sp_0)*sin(yaw_sp_0) + (y_lm_1 - y_sp_0)*cos(yaw_sp_0), (2*(x_lm_1 - x_sp_0)*cos(yaw_sp_0)**2 + (y_lm_1 - y_sp_0)*sin(2*yaw_sp_0))/(2*cos(yaw_sp_0))))**2 + sigma_fa**2*(mu_d_1 - sqrt(x_lm_1**2 - 2*x_lm_1*x_sp_0 + x_sp_0**2 + y_lm_1**2 - 2*y_lm_1*y_sp_0 + y_sp_0**2))**2)/(sigma_d**2*sigma_fa**2)
--

In [3]:
import numpy as np
import matplotlib.pyplot as plt

#Randomly generate a map and robot
def random_map(num_landmarks, num_robot_poses, sigma_d=None, sigma_fa=None):
    pos_lm = np.random.rand(num_landmarks, 2)
    pose_sensor = np.random.rand(num_robot_poses, 3)
    pose_sensor[:, 2] *= 2. * np.pi
    pose_sensor[:, 2] = 0

    dist = np.expand_dims(pos_lm, 0) - np.expand_dims(pose_sensor[:, :2], 1) #[num_robot_poses, num_landmarks, 2]

    fa = np.mod(np.arctan2(dist[:, :, 0], dist[:, :, 1]) - np.expand_dims(pose_sensor[:, -1], 1) + 2. * np.pi, 2. * np.pi)
    
    d = np.sqrt(dist[:, :, 0] ** 2 + dist[:, :, 1] ** 2)


    return pos_lm, pose_sensor, fa, d

def plot(pos_lm, pose_sensor, d, fa):
    num_robot_poses = pose_sensor.shape[0]
    num_landmarks = pos_lm.shape[0]

    plt.scatter(pos_lm[:, 0], pos_lm[:, 1])

    print (pose_sensor[:, 2])
    plt.scatter(pose_sensor[:, 0], pose_sensor[:, 1])
    for i in range(num_robot_poses):
        plt.arrow(pose_sensor[i, 0], pose_sensor[i, 1], 0.05 * np.sin(pose_sensor[i, 2]), 0.05 * np.cos(pose_sensor[i, 2]))


    for i in range(num_robot_poses):
        for j in range(num_landmarks):
            plt.arrow(pose_sensor[i, 0], pose_sensor[i, 1], d[i, j] * np.sin(fa[i,j] + pose_sensor[i, 2]), d[i,j] * np.cos(fa[i,j] + pose_sensor[i, 2]), color='red')



In [5]:
import os
parentdir = os.path.dirname(os.path.dirname(os.path.abspath('')))
parentdir = os.path.dirname(os.path.dirname(parentdir))
os.sys.path.insert(0, parentdir) 

from equations.methodology.pytorchsymbolic.sympytorch import SymPyModule
import torch

gt_pos_lm, gt_pose_sensor, fa, d = random_map(3, 2)

print ('gt_pos_lm: ', gt_pos_lm.shape)
print ('gt_pose_sensor: ', gt_pose_sensor.shape)

fa = torch.Tensor(fa)
d = torch.Tensor(d)
sigma_d = torch.Tensor([0.1])
sigma_fa = torch.Tensor([0.1])

"""
#initialize randomly
init_vals = {
    'x_lm_0': np.random.rand(),
    'y_lm_0': np.random.rand(),
    'x_lm_1': np.random.rand(),
    'y_lm_1': np.random.rand(),
    'x_lm_2': np.random.rand(),
    'y_lm_2': np.random.rand(),

    'x_sp_0': np.random.rand(),
    'y_sp_0': np.random.rand(),
    'yaw_sp_0': np.random.rand(),
    'x_sp_1': np.random.rand(),
    'y_sp_1': np.random.rand(),
    'yaw_sp_1': np.random.rand(),
}
"""

#initialize randomly
init_vals = {
    'x_lm_0': gt_pos_lm[0, 0],
    'y_lm_0': gt_pos_lm[0, 1],
    
    'x_lm_1': gt_pos_lm[1, 0],
    'y_lm_1': gt_pos_lm[1, 1],

    'x_lm_2': gt_pos_lm[2, 0],
    'y_lm_2': gt_pos_lm[2, 1],


    'x_sp_0': gt_pose_sensor[0, 0],
    'y_sp_0': gt_pose_sensor[0, 1],
    'yaw_sp_0': gt_pose_sensor[0, 2],

    'x_sp_1': gt_pose_sensor[1, 0],
    'y_sp_1': gt_pose_sensor[1, 1],
    'yaw_sp_1': gt_pose_sensor[1, 2],
}

sm = SymPyModule(eqs, init_vals=init_vals)

#for name, param in sm.named_parameters():
#    print (name, param)

optimizer = torch.optim.SGD(sm.parameters(), lr=1e-4)

for i in range(10):
    optimizer.zero_grad()
    output = sm(mu_fa_0=fa[0, 0], mu_d_0=d[0, 0], 
                mu_fa_1=fa[0, 1], mu_d_1=d[0, 1], 
                mu_fa_2=fa[0, 2], mu_d_2=d[0, 2], 
                mu_fa_3=fa[1, 0], mu_d_3=d[1, 0], 
                mu_fa_4=fa[1, 1], mu_d_4=d[1, 1], 
                mu_fa_5=fa[1, 2], mu_d_5=d[1, 2], 
                sigma_d=sigma_d, sigma_fa=sigma_fa)
    print (output)
    loss = -output.mean()

    print (loss)

    loss.backward()
    optimizer.step()

out = sm(mu_fa_0=fa[0, 0], mu_d_0=d[0, 0], 
   mu_fa_1=fa[0, 1], mu_d_1=d[0, 1], 
   mu_fa_2=fa[0, 2], mu_d_2=d[0, 2], 
   mu_fa_3=fa[1, 0], mu_d_3=d[1, 0], 
   mu_fa_4=fa[1, 1], mu_d_4=d[1, 1], 
   mu_fa_5=fa[1, 2], mu_d_5=d[1, 2], 
   sigma_d=sigma_d, sigma_fa=sigma_fa)

#for name, param in sm.named_parameters():
#    print (name, param)

#print (out)

gt_pos_lm:  (3, 2)
gt_pose_sensor:  (2, 3)
tensor([[ -327.7886,  -732.7625,  -458.3666, -1752.4690,  -351.8254,  -228.9491]],
       grad_fn=<StackBackward0>)
tensor(642.0269, grad_fn=<NegBackward0>)
tensor([[-2678.1951,  -630.2900,  -393.6974, -1652.7833,  -328.2701,  -215.5063]],
       grad_fn=<StackBackward0>)
tensor(983.1237, grad_fn=<NegBackward0>)
tensor([[-2414.3684,  -606.7716,  -376.4701, -1548.1591,  -305.5435,  -203.8596]],
       grad_fn=<StackBackward0>)
tensor(909.1954, grad_fn=<NegBackward0>)
tensor([[-2225.9692,  -587.5264,  -363.6197, -1461.5962,  -283.9068,  -193.2152]],
       grad_fn=<StackBackward0>)
tensor(852.6389, grad_fn=<NegBackward0>)
tensor([[-2082.3350,  -569.1104,  -352.8515, -1387.3137,  -263.1970,  -183.3441]],
       grad_fn=<StackBackward0>)
tensor(806.3586, grad_fn=<NegBackward0>)
tensor([[-1967.4906,  -550.0947,  -343.0348, -1321.8623,  -243.3960,  -174.1223]],
       grad_fn=<StackBackward0>)
tensor(766.6668, grad_fn=<NegBackward0>)
tensor([[-1872.